Домашнее задание № 3. Группировка данных и оконные функции (vo_HW) . Ихматуллаев Даврон

In [2]:
import pandas as pd

/tmp/ipykernel_499586/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Создание базы данных и таблиц

*Несмотря на то, что в ДЗ №3 было включены вложения transactions.csv и customers.csv, аналогично ДЗ №2. Было решено создать новую базу и таблицы на тот, случай, если данные разнятся в таблицах ДЗ№2 и ДЗ№3.

In [3]:
customer_df = pd.read_csv('customer.csv', sep=';')
customer_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [4]:
transaction_df = pd.read_csv('transaction.csv', sep=';')
transaction_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


---

#### Создадим базу данных и таблицы в Postgres при помощи DBeaver

In [ ]:
create database superstore_hw3;

create table customer_hw3
(	
	customer_id int4 primary key
	,first_name varchar(50)
	,last_name varchar(50)
	,gender varchar(30)
	,dob varchar(50)
	,job_title varchar(50)
	,job_industry_category varchar(50)
	,wealth_segment varchar(50)
	,deceased_indicator varchar(50)
	,owns_car varchar(30)
	,address varchar(50)
	,postcode varchar(30)
	,state varchar(30)
	,country varchar(30)
	,property_valuation int4
);

create table transaction_hw3

(
	transaction_id int4 primary key
	,product_id int4
	,customer_id int4
	,transaction_date varchar(30)
	,online_order varchar(30)
	,order_status varchar(30)
	,brand varchar(30)
	,product_line varchar(30)
	,product_class varchar(30)
	,product_size varchar(30)
	,list_price float4
	,standard_cost float4
	,foreign key(customer_id) references customer_hw3 (customer_id) 
);


---

#### Импортируем данные из csv в таблицы 

Данные для таблицы customers_hw3 успешно импортировалась:

<img src='../static/img/hw3_13.png'>

Однако при импорте данных в таблицу transaction_hw3 вышла ошибка о том, что в связанной таблице customers отсутствует ключ customer_id 5034

<img src='../static/img/hw2_1.png'>

Было принято решение исключить эти строки из исходной таблицы:

In [5]:
transaction_df = transaction_df.query('customer_id != 5034')

In [6]:
transaction_df.to_csv('transaction_new.csv', index=False)

Теперь импорт данных в таблицу transaction_20240101 прошел успешно:

<img src='../static/img/hw3_12.png'>

# ЗАДАНИЯ

### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)

In [ ]:
select  job_industry_category, count(*) as count_of_customers
from customer_hw3 c 
group by job_industry_category 
order by count_of_customers desc;

<img src='../static/img/hw3_1.png'>

### 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)

In [ ]:
select to_char(date_trunc('month', t.transaction_date::date), 'YYYY-MM') as transaction_month, c.job_industry_category, sum(t.list_price) 
from transaction_hw3 t
inner join customer_hw3 c 
on t.customer_id = c.customer_id 
group by c.job_industry_category , transaction_month
order by transaction_month, c.job_industry_category;

<img src='../static/img/hw3_2.png'>

### 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)

In [ ]:
select t.brand , count(*)
from transaction_hw3 t 
inner join customer_hw3 c 
on t.customer_id = c.customer_id 
where c.job_industry_category = 'IT'  and t.online_order = 'True' and t.order_status = 'Approved'
group by t.brand 
order by t.brand desc;

<img src='../static/img/hw3_3.png'>

### 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. — (2 балла)

 1й способ с group by

In [ ]:
select  customer_id , 
		sum(list_price) as sum_of_transactions, 
		max(list_price) as max_of_transactions,
		min(list_price) as min_of_transactions,
		count(transaction_id) as transaction_count
from transaction_hw3 t 
group by customer_id
order by sum(list_price) desc, count(transaction_id) desc;

<img src='../static/img/hw3_4.png'>

 2 способ с оконными функциями

In [ ]:
select  distinct 
		customer_id,
		sum(list_price) over(partition by customer_id) as sum_of_transactions,
		max(list_price) over(partition by customer_id) as max_of_transactions,
		min(list_price) over(partition by customer_id) as min_of_transactions,
		count(transaction_id) over(partition by customer_id) as transaction_count
from transaction_hw3 t 
order by sum_of_transactions desc, transaction_count desc

<img src='../static/img/hw3_5.png'>

### 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы. — (2 балла)

Клиент с минимальной суммой транзакций:

In [ ]:
with grouped_table as
(
select c.customer_id, c.first_name , c.last_name , sum(t.list_price) as sum_of_transactions
from transaction_hw3 t 
inner join customer_hw3 c 
on t.customer_id = c.customer_id 
group by c.customer_id, c.first_name , c.last_name
)

select *
from grouped_table
where sum_of_transactions = (select min(sum_of_transactions) from grouped_table);

<img src='../static/img/hw3_9.png'>

*Был использован такой запрос (без использования limit 1) для того случая, когда клиентов с минимально/максимальной суммой транзакций может быть несколько.

Клиент с максимальной суммой транзакций:

In [ ]:
with grouped_table as
(
select c.customer_id, c.first_name , c.last_name , sum(t.list_price) as sum_of_transactions
from transaction_hw3 t 
inner join customer_hw3 c 
on t.customer_id = c.customer_id 
group by c.customer_id, c.first_name , c.last_name
)

select *
from grouped_table
where sum_of_transactions = (select max(sum_of_transactions) from grouped_table);

<img src='../static/img/hw3_10.png'>

### 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)

In [ ]:
with window_table as 
(
select  customer_id
		,transaction_date::date
		,transaction_id
		,ROW_NUMBER() over(partition by customer_id order by transaction_date::date asc) as row
from transaction_hw3 t
)
select customer_id, transaction_date, transaction_id
from window_table
where row = 1;

<img src='../static/img/hw3_8.png'>

### Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).

In [ ]:
with grouped_table as 
(
with window_table as
(
select  c.customer_id, c.first_name, c.last_name, c.job_title
		,t.transaction_date::date
		,lead(t.transaction_date::date) over(partition by c.first_name, c.last_name order by t.transaction_date::date) as next_transaction_date
from transaction_hw3 t 
inner join customer_hw3 c 
on t.customer_id = c.customer_id
)

select *,
		max(next_transaction_date - transaction_date) as max_days_between
from window_table
where next_transaction_date - transaction_date notnull
group by customer_id, first_name, last_name, job_title, transaction_date, next_transaction_date
order by max_days_between desc
)

select *
from grouped_table
where max_days_between = (select max(max_days_between) from grouped_table);

*Был использован такой запрос (без использования limit 1) для того случая, когда клиентов с максимальным интервалом транзакций может быть несколько.

<img src='../static/img/hw3_11.png'>